# Toy Problem - Transformer Application to Regression

https://github.com/oliverguhr/transformer-time-series-prediction/tree/master

Description: This example is from the repo above. It contains 2 PyTorch models for a transformer-based time series prediction. The dataset is stored in ./daily-min-temperatures.csv

In [1]:
import torch 
import torch.nn as nn
import numpy as np 
import time 
import math
from matplotlib import pyplot

In [2]:
torch.manual_seed(0)
np.random.seed(0)

In [3]:
calculate_loss_over_all_values = False

In [4]:
# S = source sequence length
# T = target sequence length 
# N = batch size 
# E = feature number

In [5]:
input_window = 100 
output_window = 5
batch_size = 10 
lr = 0.005
epochs = 100
device = torch.device("curda" if torch.cuda.is_available() else "cpu")

In [6]:
class PositionalEncoding(nn.Module): # define a pytorch module that inherits nn.Module
    """
    Positional encoding layer for transformer model. 

    Layer injects info about the relative or absolute position of the sequence, without adding learnable parameters. 

    Uses sin and cos fcns of different frequencies to encode position info. 

    Args: 
        d_model (int): dimension of the embedding space 
        max_len (int, optional): max sequence length supported. Default is 5000 

    Attriutes: 
        pe (Tensor): Fixed positional encoding matrix of space (max_len, 1, d_model)
    
    """

    # the "Attributes" part documents the instance variables inside the __init__

    def __init__(self, d_model, max_len=5000): # creates init method 
        super(PositionalEncoding, self).__init__() # super() lets us avoid referring to the base class explicitly
        # https://stackoverflow.com/questions/576169/understanding-python-super-with-init-methods
        pe = torch.zeros(max_len, d_model) # create empty matrix of shape max_len X d_model to hold the positional encodings
        # row: position i.e. 0, 1, 2, 
        # column: dim of the embedding 

        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1) # vector of positions [0, 1, 2, 3, ..., 4999]
        # unsqueeze reshapes vector from [max_len,] to [max_len, 1] to enable broadcasting 

        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        # division term for the sin and cos fcns 
        # torch.arange(0, d_model, 2).float(): starts at 0, ends at d_model, step size = 2 
        # -ln(10000)
        # torch.exp = exp 
        # this comes from "Attention is all you need" paper where sin and cos fcns of different frequencies are used where each dimension of the positional encoding corresponds to a sine
        # PE at dim i = PE_(pos, 2i) = sin(pos/10000^(2i/d_model))
        # PE_(pos, 2i+1) = cos(pos/10000^(2i/d_model)) 

        pe[:, 0::2] = torch.sin(position * div_term)
        # at even indices: sin(position * frequency)
    
        pe[:, 1::2] = torch.cos(position * div_term)
        # at odd indices: cos(position * frequency)

        pe = pe.unsqueeze(0).transpose(0,1)
        # pe.unsqueeze(0) == adds a batch dimension so the shape becomes: [1, max_len, d_model]
        # .transpose(0,1) == swaps the first and second dimensions such that the new shape is [max_len, 1, d_model]

        self.register_buffer('pe', pe)
        # saving pe tensor. Tensor which is not a parameter, but should be part of the module's state. Used for tensors that need to be on the same device as the module. 
        # it's a fixed tensor stored with the model and moved to the GPU/CPU automatically 
        # this is NOT updated during backprop 

    def forward(self, x): # during the forward pass, x is the input with shape [sequence length, batch_size, d_model]
        return x + self.pe[:x.size(0), :] # add the pe for the len of the input, x 

In [7]:
# EXAMPLE USE OF POSITIONAL ENCODING
positional_encoder = PositionalEncoding(d_model = 512)
sample_x = torch.randn(100, 32, 512) # tensor filled with random numbers from a standard normal distribution of shape [100, 32, 512]
sample_encode = positional_encoder(sample_x)

In [8]:
print(sample_x[0:3, 0:3, 0:3])

tensor([[[-1.1258, -1.1524, -0.2506],
         [-0.5461, -0.6302, -0.6347],
         [-1.0841, -0.1287, -0.6811]],

        [[-0.5518,  1.5398,  1.0036],
         [-0.4424,  0.2087,  0.0160],
         [ 1.2970, -0.4725,  0.3149]],

        [[-0.9780,  0.6038, -1.7178],
         [-0.3399, -0.2990,  1.8007],
         [ 0.6786,  0.5225, -0.0246]]])


In [9]:
print(sample_encode[0:3, 0:3, 0:3])

tensor([[[-1.1258, -0.1524, -0.2506],
         [-0.5461,  0.3698, -0.6347],
         [-1.0841,  0.8713, -0.6811]],

        [[ 0.2896,  2.0801,  1.8254],
         [ 0.3990,  0.7490,  0.8379],
         [ 2.1385,  0.0678,  1.1368]],

        [[-0.0687,  0.1877, -0.7814],
         [ 0.5694, -0.7151,  2.7371],
         [ 1.5879,  0.1063,  0.9118]]])


In [10]:
class TransAm(nn.Module): # defines a model class 
    def __init__(self, feature_size=250, num_layers=1, dropout=0.1): # initialize model 
        # feature size: input embedding dimension 
        # num_layers: transformer layers to stack 
        # dropout: dropout rate for regularization 

        super(TransAm, self).__init__()
        self.model_type = 'Transformer' # string label 
        self.src_mask = None # used for sequence masking - important in autoregressive tasks like time-series and language modeling
        self.pos_encoder = PositionalEncoding(feature_size) # giving position awareness to the input embeddings before processed by the transformer
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=feature_size, nhead=10, dropout=dropout) # one transformer encoder layer with 10 heads 
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers) # stack multiple copies of the encoder layer 
        self.decoder = nn.Linear(feature_size, 1) # maps the feature size to 1 (in time series forecasting, this maps one number per position)
        self.init_weights() # calls init_weights methods to initialize the weights 

    def init_weights(self): 
        initrange = 0.1 # define a small numer to set range for random initialization 
        self.decoder.bias.data.zero_() # bias is the additive constant inside Linear layers. This overwrites all biases to 0 so that only weights matter. Random biases could introduce unwanted drift right at the start so we initialize at 0
        self.decoder.weight.data.uniform_(-initrange, initrange) # initialize weights between -0.1 and 0.1. Smaller weights help stabilize training early on 

    def forward(self,src): # the forward pass for the model 

        # because this is a causal task (like time series or language modeling), we must prevent tokens from seeing the future, so we implement this mask to avoid attention to future tokens
        # the mask is a square matrix of size (seq_length, seq_length)


        if self.src_mask is None or self.src_mask.size(0) !=len(src): # check if a new source mask needs to be created (or a new one)
            # if self.src_mask is None then no mask was created yet
            #if self.src_mask.size(0) != len(src), the input seq len has changed since the previous iteration therefore we need one of a new size 

            device = src.device # get the device where the input tensor lives to ensure the mask is on the same device 
            mask = self._generate_square_subsequent_mask(len(src)).to(device) 
            # generate causal mask of len(src) such that the token can attend only to itself and earlier tokens
            self.src_mask = mask 

        src = self.pos_encoder(src) # apply positional encoding to the input embeddings 
        output = self.transformer_encoder(src, self.src_mask) # pass position-encoded input into the stacked transformer encoder layers 
        # this is where mlti-head self-attention happens 
        # each token attends to the previous ones bc of the mask 

        output = self.decoder(output)
        # apply Linear layer to every position 
        # turn the feature_size vector to a scalar 
        # this is the model's final prediction at each time step or token 

        return output

    def _generate_square_subsequent_mask(self, sz): 
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0,1) # create upper triangular matrix of ones, then trnaspose flips it to make lower triangle of 1s 
        mask = mask.float().masked_fill(mask==0, float('-inf')).masked_fill(mask==1, float(0.0)) # convert 1s/0s into attention scores. mask == 0 ==> future positions, mask == 1 ==> self and past 
        return mask

In [11]:
# function to prepare sequence data for training model

def create_inout_sequences(input_data, tw): 
    # function: slices a time series dataset into overlapping i/o sequences for training 
    # input_data: 1D array (the time series)
    # tw: time window (the sequence length)
    # output: list of (input sequence, label sequence) pairs 

    inout_seq = [] # preallocate memory to hold the (input, label) pairs 
    L = len(input_data) # stores the length of the sequence 
    for i in range(L-tw): # loop through the sequence to get all sliding windows of length tw
        # stop at L-tw to ensure input_data[i:i+tw] stays within the length of the input_data 

        train_seq = np.append(input_data[i:i+tw][:-output_window], output_window * [0])
        # input_data[i:i+tw]: gives a window of length tw 
        # [:-output_window]: removes the last output_window values - therefore we're making the last value(s) we want the model to predict 
        # output_window * [0]: appends 0s at the end for the same length removed

        train_label = input_data[i:i+tw]
        # this is the ground truth - the full unmasked windwo including the parts zeroed out 

        inout_seq.append((train_seq, train_label))
        # saves tuple of the masked input and the ground label 

    return torch.FloatTensor(inout_seq) # convert the tuple into a PyTorch tensor 

In [ ]:
# function generates and prepares synthetic time series data for the transformer, including normalizaton and splitting into training and tests sets 
def get_data(): 

    # generate synthetic data of a continuous signal 
    time = np.arange(0, 400, 0.1) # len = 4000 
    amplitude = np.sin(time) + np.sin(time * 0.05) + np.sin(time * 0.12) * np.random.normal(-0.2, 0.2, len(time))
    # np.sin(time): main wave 
    # np.sin(time*0.05): low-frequency drift
    # np.sin(time*0.12): modulated noise 

    # scaling the data 
    from sklearn.preprocessing import MinMaxScaler 
    scaler = MinMaxScaler(feature_range=(-1, 1))
    amplitude = scaler.fit_transform(amplitude.reshape(-1, 1)).reshape(-1)
    # reshape(-1, 1): turns a 1D array into a 2D shape expected by MinMaxScaler, then .reshape(-1) flattens it again 

    # data splitting 
    samples = 2800 # 70% of 4000
    train_data = amplitude[:samples]
    test_data = amplitude[samples:]

    train_sequence = create_inout_sequences(train_data, input_window)
    train_sequence = train_sequence[:-output_window]

    test_data = create_inout_sequences(test_data, input_window)
    test_data = test_data[:-output_window]

    return train_sequence.to(device), test_data.to(device)

In [ ]:
def get_batch(source, i, batch_size): # extracts a mini-batch of input/label pairs from the training or test set (the source), starting at position i, and prepares the right shape for the model input
    seq_len = min(batch_size, len(source) - 1 - i) # calculates how many items to include in the current batch - if you're near the end of the dataset, you may not have enough for a full batch so you'll take whatever is left
    data = source[i:i+seq_len] # slices the dataset from i to i+seq_len
    input = torch.stack(torch.stack([item[0] for item in data]).chunk(input_window, 1))
    target = torch.stack(torch.stack([item[1] for item in data]).chunk(input_window, 1))
    return input, target

In [ ]:
def train(train_data): 
    # define the train function, takes in train_data, process it in batches to train the model for one epoch
    model.train() # sets the mode to training mode 
    # this enables layers like Dropout and BatchNorm to behave correctly - as opposed to how they do during .eval() mode

    total_loss = 0.0 # initialize loss counter 
    start_time = time.time() # starts timer at start of training  - used to calculate how long each batch takes

    for batch, i in enumerate(range(0, len(train_data) - 1, batch_size)):
        # enumerate gives: batch size index, the start index of the train_data

        data, targets = get_batch(train_data, i, batch_size)
        # extracts a batch of inputs starting at index i 

        optimizer.zero_grad()
        # clear old gradients from previous step 

        output = model(data)
        # feeds input batch into the model to store the predictions

        if calculate_loss_over_all_values: # uses entire sequence to calculate loss
            loss = criterion(output, targets)
        else: # uses only the previous output window of timesteps to calculate loss
            loss = criterion(output[-output_window:], targets[-output_window:])

        loss.backward() # compute gradients of loss wrt the learnable parameters
        # backward pass 

        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5) # clip gradients if norm exceeds 0.5 to prevent exploding gradients 
        optimizer.step() # apply gradients to update model weights 
        # this is where learning occurs 

        total_loss += loss.item() # add current loss to cummulative loss
        log_interval = int(len(train_data) / batch_size / 5)

        if batch % log_interval == 0 and batch > 0:
            cur_loss = total_loss / log_interval 
            elapsed = time.time() - start_time 
            print ('| epoch {:3d} | {:5d}/{:5d} batches | '
                    ' lr {:02.6f} | {:5.2f} ms | loss {:5.5f} | ppl {:8.2f}'.format(
                        epoch, batch, len(train_data) // batch_size, scheduler.get_lr()[0],
                        elapsed * 1000 / log_interval, cur_loss, math.exp(cur_loss)))
            # ppl = perplexity
            total_loss = 0 
            start_time = time.time()

In [ ]:
def plot_and_loss(eval_model, data_source, epoch): 
    # evaluates model on the dataset, data_source 

    eval_model.eval() # sets model to evaluatio model 
    total_loss = 0.0 # accumulated loss across test examples 
    test_result = torch.Tensor(0) # stores model outputs for plotting 
    truth = torch.Tensor(0) # ground truth values

    #  torch.Tensor(0): creates empty 1D tensors 

    with torch.no_grad(): # opens the no-gradient context 
        # during evaluation, we don't need gradients 

        for i in range(0, len(data_source) - 1): # iterate over every sample of data_source except the last
            data, target = get_batch(data_source, i, 1)
            output = eval_model(data)
            if calculate_loss_over_all_values: 
                total_loss += criterion(output, target).item()
            else: 
                total_loss += criterion(output[-output_window:], target[-output_window:]).item()

            test_result = torch.cat((test_result, output[-1].view(-1).cpu()), 0)
            truth = torch.cat((truth, target[-1].view(-1).cpu()), 0)
    
    len(test_result)

    pyplot.plot(test_result, color = "red") # model predictions 
    pyplot.plot(truth[:500], color = "blue") # true values 
    pyplot.plot(test_result - truth, color = "green") # error
    pyplot.grid(True, which = 'both')
    pyplot.axhline(y=0, color='k')
    pyplot.savefig('graph/transformer-epoch%d.png' % epoch)
    pyplot.close()

    return total_loss/i

In [ ]:
def predict_future(eval_model, data_source, steps): 
    #  enables trained transformer to predict future time steps beyond the training window 
    eval_model.eval() # puts model in evaluation mode 
    total_loss = 0.0
    test_result = torch.Tensor(0)
    truth = torch.Tensor(0)
    _, data = get_batch(data_source, 0, 1) # obtain single sample from dataset to use as the initial input sequence 

    with torch.no_grad():
        for i in range(0, steps, 1): 
            input = torch.clone(data[-input_window:]) # create copy of last input_window timesteps from data
            input[-output_window:] = 0 # zero out the last output_window values in the input. Simulates missing future steps for the model to predict 
            output = eval_model(data[-input_window:]) 
            data = torch.cat((data, output[-1:]))

        data = data.cpu().view(-1)

        pyplot.plot(data, color = 'red') # full series (original and predicted)
        pyplot.plot(data[:input_window], color = 'blue') # initial known input 
        pyplot.grid(True, which = 'both') 
        pyplot.axhline(y=0, color='k')
        pyplot.savefig('graph/transformer-future%d.png' % steps)
        pyplot.close()

In [ ]:
def evaluate(eval_model, data_source): 
    # defines function to evaluate the model's loss on the data_source dataset - this is called after each epoch for the validation set 
    eval_model.eval() # set model to evaluation mode
    total_loss = 0.0 # initialize total loss
    eval_batch_size = 1000 # set large batch size since we're not doing backprop 
    with torch.no_grad(): # being no-gradient context 
        for i in range(0, len(data_source) - 1, eval_batch_size):
            data, targets = get_batch(data_source, i, eval_batch_size)
            output = eval_model(data)

            if calculate_loss_over_all_values: 
                total_loss += len(data[0]) * criterion(output, targets).cpu().item()
            else: 
                total_loss += len(data[0]) * criterion(output[-output_window:], targets[-output_window:]).cpu().item()
            
    return total_loss / len(data_source)

In [ ]:
train_data, val_data = get_data() 
#  data preprocessing function 

model = TransAm().to(device)
# instantiate transformer model 

criterion = nn.MSELoss()
# define loss function 

optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
# set up optimizer (Adam with decoupled weight decay)

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma = 0.98)
# create learning rate scheduler 
# at every step size = 1, epoch, it multiples the LR by gamma=98 to encourage more stable convergence

best_val_loss = float("inf") # track the lowest validation loss
best_model = None # best performing model 

for epoch in range(1, epochs + 1): # training loop from epoch = 1
    epoch_start_time = time.time()  # start time for logging 
    train(train_data) # train model 

    if (epoch % 10 == 0): # every 10 epochs 
        val_loss = plot_and_loss(model, val_data, epoch) # compute val loss and plot model performance
        predict_future(model, val_data, 200) # forecast 200 steps in future
    else: 
        val_loss = evaluate(model, val_data)

    print('-' * 89)
    print('| end of epoch {:3d} | time: {:5.2}s | valid loss {:5.5f} | valid ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time), val_loss, math.exp(val_loss)))
    print('-' * 89)

    scheduler.step()

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import time
import math
from matplotlib import pyplot

torch.manual_seed(0)
np.random.seed(0)

# This concept is also called teacher forceing. 
# The flag decides if the loss will be calculted over all 
# or just the predicted values.
calculate_loss_over_all_values = False

# S is the source sequence length
# T is the target sequence length
# N is the batch size
# E is the feature number

#src = torch.rand((10, 32, 512)) # (S,N,E) 
#tgt = torch.rand((20, 32, 512)) # (T,N,E)
#out = transformer_model(src, tgt)
#
#print(out)

input_window = 100
output_window = 5
batch_size = 10 # batch size
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class PositionalEncoding(nn.Module):

    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()       
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        #pe.requires_grad = False
        self.register_buffer('pe', pe)

    def forward(self, x):
        return x + self.pe[:x.size(0), :]
       

class TransAm(nn.Module):
    def __init__(self,feature_size=250,num_layers=1,dropout=0.1):
        super(TransAm, self).__init__()
        self.model_type = 'Transformer'
        
        self.src_mask = None
        self.pos_encoder = PositionalEncoding(feature_size)
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=feature_size, nhead=10, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)        
        self.decoder = nn.Linear(feature_size,1)
        self.init_weights()

    def init_weights(self):
        initrange = 0.1    
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self,src):
        if self.src_mask is None or self.src_mask.size(0) != len(src):
            device = src.device
            mask = self._generate_square_subsequent_mask(len(src)).to(device)
            self.src_mask = mask

        src = self.pos_encoder(src)
        output = self.transformer_encoder(src,self.src_mask)#, self.src_mask)
        output = self.decoder(output)
        return output

    def _generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask



# if window is 100 and prediction step is 1
# in -> [0..99]
# target -> [1..100]
def create_inout_sequences(input_data, tw):
    inout_seq = []
    L = len(input_data)
    for i in range(L-tw):
        train_seq = np.append(input_data[i:i+tw][:-output_window] , output_window * [0])
        train_label = input_data[i:i+tw]
        #train_label = input_data[i+output_window:i+tw+output_window]
        inout_seq.append((train_seq ,train_label))
    return torch.FloatTensor(inout_seq)

def get_data():
    # time        = np.arange(0, 400, 0.1)
    # amplitude   = np.sin(time) + np.sin(time*0.05) +np.sin(time*0.12) *np.random.normal(-0.2, 0.2, len(time))
    
    from pandas import read_csv
    series = read_csv('daily-min-temperatures.csv', header=0, index_col=0, parse_dates=True)
    # series = read_csv('daily-min-temperatures.csv', header=0, index_col=0, parse_dates=True, squeeze=True)
    
    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler(feature_range=(-1, 1)) 
    amplitude = scaler.fit_transform(series.to_numpy().reshape(-1, 1)).reshape(-1)
    # amplitude = scaler.fit_transform(amplitude.reshape(-1, 1)).reshape(-1)
    
    
    sampels = 2800
    train_data = amplitude[:sampels]
    test_data = amplitude[sampels:]

    # convert our train data into a pytorch train tensor
    #train_tensor = torch.FloatTensor(train_data).view(-1)
    # todo: add comment.. 
    train_sequence = create_inout_sequences(train_data,input_window)
    train_sequence = train_sequence[:-output_window] #todo: fix hack?

    #test_data = torch.FloatTensor(test_data).view(-1) 
    test_data = create_inout_sequences(test_data,input_window)
    test_data = test_data[:-output_window] #todo: fix hack?

    return train_sequence.to(device),test_data.to(device)

def get_batch(source, i,batch_size):
    seq_len = min(batch_size, len(source) - 1 - i)
    data = source[i:i+seq_len]    
    input = torch.stack(torch.stack([item[0] for item in data]).chunk(input_window,1)) # 1 is feature size
    target = torch.stack(torch.stack([item[1] for item in data]).chunk(input_window,1))
    return input, target


def train(train_data):
    model.train() # Turn on the train mode
    total_loss = 0.
    start_time = time.time()

    for batch, i in enumerate(range(0, len(train_data) - 1, batch_size)):
        data, targets = get_batch(train_data, i,batch_size)
        optimizer.zero_grad()
        output = model(data)        

        if calculate_loss_over_all_values:
            loss = criterion(output, targets)
        else:
            loss = criterion(output[-output_window:], targets[-output_window:])
    
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        total_loss += loss.item()
        log_interval = int(len(train_data) / batch_size / 5)
        if batch % log_interval == 0 and batch > 0:
            cur_loss = total_loss / log_interval
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | '
                  'lr {:02.6f} | {:5.2f} ms | '
                  'loss {:5.5f} | ppl {:8.2f}'.format(
                    epoch, batch, len(train_data) // batch_size, scheduler.get_lr()[0],
                    elapsed * 1000 / log_interval,
                    cur_loss, math.exp(cur_loss)))
            total_loss = 0
            start_time = time.time()

def plot_and_loss(eval_model, data_source,epoch):
    eval_model.eval() 
    total_loss = 0.
    test_result = torch.Tensor(0)    
    truth = torch.Tensor(0)
    with torch.no_grad():
        for i in range(0, len(data_source) - 1):
            data, target = get_batch(data_source, i,1)
            # look like the model returns static values for the output window
            output = eval_model(data)    
            if calculate_loss_over_all_values:                                
                total_loss += criterion(output, target).item()
            else:
                total_loss += criterion(output[-output_window:], target[-output_window:]).item()
            
            test_result = torch.cat((test_result, output[-1].view(-1).cpu()), 0) #todo: check this. -> looks good to me
            truth = torch.cat((truth, target[-1].view(-1).cpu()), 0)
            
    #test_result = test_result.cpu().numpy()
    len(test_result)

    pyplot.plot(test_result,color="red")
    pyplot.plot(truth[:500],color="blue")
    pyplot.plot(test_result-truth,color="green")
    pyplot.grid(True, which='both')
    pyplot.axhline(y=0, color='k')
    pyplot.savefig('graph/transformer-epoch%d.png'%epoch)
    pyplot.close()
    
    return total_loss / i


def predict_future(eval_model, data_source,steps):
    eval_model.eval() 
    total_loss = 0.
    test_result = torch.Tensor(0)    
    truth = torch.Tensor(0)
    _ , data = get_batch(data_source, 0,1)
    with torch.no_grad():
        for i in range(0, steps,1):
            input = torch.clone(data[-input_window:])
            input[-output_window:] = 0     
            output = eval_model(data[-input_window:])                        
            data = torch.cat((data, output[-1:]))
            
    data = data.cpu().view(-1)
    

    pyplot.plot(data,color="red")       
    pyplot.plot(data[:input_window],color="blue")
    pyplot.grid(True, which='both')
    pyplot.axhline(y=0, color='k')
    pyplot.savefig('graph/transformer-future%d.png'%steps)
    pyplot.close()
        
# entweder ist hier ein fehler im loss oder in der train methode, aber die ergebnisse sind unterschiedlich 
# auch zu denen der predict_future
def evaluate(eval_model, data_source):
    eval_model.eval() # Turn on the evaluation mode
    total_loss = 0.
    eval_batch_size = 1000
    with torch.no_grad():
        for i in range(0, len(data_source) - 1, eval_batch_size):
            data, targets = get_batch(data_source, i,eval_batch_size)
            output = eval_model(data)            
            if calculate_loss_over_all_values:
                total_loss += len(data[0])* criterion(output, targets).cpu().item()
            else:                                
                total_loss += len(data[0])* criterion(output[-output_window:], targets[-output_window:]).cpu().item()            
    return total_loss / len(data_source)

train_data, val_data = get_data()
model = TransAm().to(device)

criterion = nn.MSELoss()
lr = 0.005 
#optimizer = torch.optim.SGD(model.parameters(), lr=lr)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.98)

best_val_loss = float("inf")
epochs = 100 # The number of epochs
best_model = None

for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train(train_data)
    
    
    if(epoch % 10 is 0):
        val_loss = plot_and_loss(model, val_data,epoch)
        predict_future(model, val_data,200)
    else:
        val_loss = evaluate(model, val_data)
        
    print('-' * 89)
    print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.5f} | valid ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time),
                                     val_loss, math.exp(val_loss)))
    print('-' * 89)

    #if val_loss < best_val_loss:
    #    best_val_loss = val_loss
    #    best_model = model

    scheduler.step() 

#src = torch.rand(input_window, batch_size, 1) # (source sequence length,batch size,feature number) 
#out = model(src)
#
#print(out)
#print(out.shape)

<>:257: SyntaxWarning: "is" with 'int' literal. Did you mean "=="?
<>:257: SyntaxWarning: "is" with 'int' literal. Did you mean "=="?
/var/folders/jm/d49dqhd91j9g3f2wf9zhq0z80000gn/T/ipykernel_13047/3125818542.py:95: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /private/var/folders/k1/30mswbxs7r1g6zwn8y4fyt500000gp/T/abs_2634bauad6/croot/libtorch_1744642078920/work/torch/csrc/utils/tensor_new.cpp:281.)
  return torch.FloatTensor(inout_seq)
/Users/marilyn/anaconda3/envs/toy_transformer/lib/python3.13/site-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(
/var/folders/jm/d49dqhd91j9g3f2wf9zhq0z80000gn/T/ipykernel_13047/3125

| epoch   1 |    53/  269 batches | lr 0.005000 | 440.53 ms | loss 5.30309 | ppl   200.96
| epoch   1 |   106/  269 batches | lr 0.005000 | 467.20 ms | loss 0.08589 | ppl     1.09
| epoch   1 |   159/  269 batches | lr 0.005000 | 444.79 ms | loss 0.12388 | ppl     1.13
| epoch   1 |   212/  269 batches | lr 0.005000 | 434.17 ms | loss 0.05552 | ppl     1.06
| epoch   1 |   265/  269 batches | lr 0.005000 | 416.57 ms | loss 0.05051 | ppl     1.05
-----------------------------------------------------------------------------------------
| end of epoch   1 | time: 120.31s | valid loss 0.11385 | valid ppl     1.12
-----------------------------------------------------------------------------------------
| epoch   2 |    53/  269 batches | lr 0.004802 | 460.81 ms | loss 0.06192 | ppl     1.06
| epoch   2 |   106/  269 batches | lr 0.004802 | 446.98 ms | loss 0.07375 | ppl     1.08
| epoch   2 |   159/  269 batches | lr 0.004802 | 446.87 ms | loss 0.07587 | ppl     1.08
| epoch   2 |   212/  2